## Google Scholar Crawler
*DISCLAIMER: after retrieving 490 URLs, Google BLOCKED the IP address

*TODO: add selenium and add a manual step for opening a url and showing that we are NOT a robot

In [1]:
keyword_raw = "flowspec"

In [2]:
import cfscrape

from lxml import etree

import pandas as pd

import urllib

import re

from random import randint
from time import sleep

In [3]:
page = 1 
df = pd.DataFrame()
    
while True: 
    start_page = (page - 1) * 10
    #################################################################################
    keyword = urllib.parse.quote_plus("\""+keyword_raw+"\"")
    url="https://scholar.google.nl/scholar?start="+str(start_page)+"&q="+keyword+"&hl=nl&as_sdt=0,5"
    print(page, url)
    #################################################################################
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content
    # print(scraped_html.decode("utf-8") )
    html = etree.HTML(scraped_html)
    #################################################################################    
    elements = html.xpath("//div[@class='gs_ri']")
    #################################################################################
    if len(elements) == 0:
        print("The code stopped for some reason!! Possibly Google blocked us!")
        print("\""+(''.join(html.xpath("//div[@id='infoDiv']/text()")))+"\"")
        break
    #################################################################################
    for element in elements:
        #################################################################################
        if len(element.xpath("h3[@class='gs_rt']/a/text()")) > 1:
            title = (keyword_raw+" ").join(element.xpath("h3[@class='gs_rt']/a/text()"))
        elif len(element.xpath("h3[@class='gs_rt']/a/text()")) == 1:
            title = element.xpath("h3[@class='gs_rt']/a/text()")[0]
        else:
            title = ""
        #################################################################################    
        link  = ''.join(element.xpath("h3[@class='gs_rt']/a/@href"))
        #################################################################################    
        abstract = ''.join(element.xpath("div[@class='gs_rs']/text()")).replace('\n','')
        #################################################################################
        reference = ''.join(element.xpath("div[@class='gs_a']/text()"))
        #################################################################################
        try:
            year = re.search(r"\d{4}", reference)[0]
        except:
            year = ""
        #################################################################################   
        df = df.append({'year':year,'title':title,'abstract':abstract,'reference':reference,'link':link},ignore_index=True)
    #################################################################################   
    print("sleeping...")
    sleep(randint(6,18))
    #################################################################################   
    page = page + 1       

1 https://scholar.google.nl/scholar?start=0&q=%22flowspec%22&hl=nl&as_sdt=0,5
The code stopped for some reason!! Possibly Google blocked us!

Deze pagina wordt weergegeven wanneer Google verzoeken detecteert die afkomstig zijn van uw computernetwerk en die de  lijken te schenden. De blokkering verloopt kort nadat die verzoeken zijn gestopt. Dit verkeer kan zijn verzonden door schadelijke software, een invoegtoepassing van de browser of een script dat automatische verzoeken verzendt. Als u uw netwerkverbinding deelt, vraagt u uw beheerder om hulp. Het is mogelijk dat een andere computer die hetzelfde IP-adres gebruikt, verantwoordelijk is voor de verzoeken. Soms kan deze pagina worden weergegeven als u geavanceerde termen gebruikt die vaak door robots worden gebruikt of als de verzoeken zeer snel worden verzonden.




## Exporting the output to a csv file

In [ ]:
df.to_csv('output_crawler_google.csv')

## Further analysis

In [25]:
pd.options.display.max_colwidth = 100
from urllib.parse import urlparse

In [36]:
#Reading again the file in case it is a offline analysis
df = pd.read_csv('output_crawler_google.csv',index_col=0).dropna()

In [37]:
df.head()

,abstract,link,reference,title,year
0,This document is a product of the Internet Engineering Task Force (IETF). It represents the cons...,http://www.rfc-editor.org/rfc/pdfrfc/rfc7674.txt.pdf,J Haas - 2015 - rfc-editor.org,Clarification of the flowspec Redirect Extended Community,2015.0
1,Effective service chaining enforcement along TE paths is proposed using Segment Routing and exte...,https://www.osapublishing.org/abstract.cfm?uri=OFC-2017-W4J.3,", , … - Optical Fiber …, 2017 - osapublishing.org",Service chaining in multi-layer networks using segment routing and extended BGP,2017.0
2,The BERKOM multimedia transport service provides a communication platform for audiovisual commun...,https://ieeexplore.ieee.org/abstract/document/369006/,"S Damaskos, - Proceedings of ICC/SUPERCOMM'94 …, 1994 - ieeexplore.ieee.org",The BERKOM-II flowspec and its Mapping on ATM,1994.0
3,Page 1. Leonardo Serodio leonardo.serodio@alcatel-lucent.com May 2013 Traffic DiversionTechnique...,https://www.nanog.org/sites/default/files/wed.general.trafficdiversion.serodio.10.pdf,L Serodio - 2013 - nanog.org,Traffic diversion techniques for DDoS mitigation using BGP,2013.0
4,"… services together. It defines the usage and contents of three RSVP protocol objects,the , ADSP...",http://www.rfc-editor.org/rfc/pdfrfc/rfc2210.txt.pdf,J Wroclawski - 1997 - rfc-editor.org,The use of RSVP with IETF integrated services,1997.0


In [38]:
print("TOTAL ENTRIES:",len(df))

TOTAL ENTRIES: 398


In [39]:
source_distribution = df['link'].dropna().apply(lambda x: urlparse(x)[1].replace("www.","")).value_counts()
print("Distinct Source of Information:",len(source_distribution))
print()
print("Frequency of Source of Information:")
print(source_distribution.head(30))

Distinct Source of Information: 80

Frequency of Source of Information:
ieeexplore.ieee.org            74
patents.google.com             51
link.springer.com              40
rfc-editor.org                 34
tools.ietf.org                 21
citeseerx.ist.psu.edu          20
spiedigitallibrary.org          9
pdfs.semanticscholar.org        9
researchgate.net                9
ipsj.ixsq.nii.ac.jp             8
books.google.nl                 8
shiraz.levkowetz.com            8
dbpia.co.kr                     7
dl.acm.org                      6
sciencedirect.com               6
buildbot.tools.ietf.org         5
academia.edu                    5
apps.dtic.mil                   3
onlinelibrary.wiley.com         3
datatracker.ietf.org            3
ftp.nara.wide.ad.jp             2
nanog.org                       2
portaldeconteudo.sbc.org.br     2
watersprings.org                2
ftp.isi.edu                     2
degruyter.com                   2
paper.edu.cn                    2
ejournal.o

The most important source of information are: 
- ieeexplore.ieee.org
- dl.acm.org
- link.springer.com
- pdfs.semanticscholar.org
- sciencedirect.com
- researchgate.net

Of course the RFCs of IETF are also very important:
- rfc-editor.org 
- tools.ietf.org
- datatracker.ietf.org

## Exporting information to csv files based on the source of information

In [40]:
df[df['link'].str.contains("ieeexplore.ieee.org")].to_csv("flowspec_ieee.csv")

In [41]:
df[df['link'].str.contains("link.springer.com")].to_csv("flowspec_springer.csv")

In [42]:
df[df['link'].str.contains("pdfs.semanticscholar.org")].to_csv("flowspec_semanticscholar.csv")

In [45]:
df[df['link'].str.contains("dl.acm.org")].to_csv("flowspec_acm.csv")

## Lessons learned:
- remove papers that have in the title or description/abstract the list of words: [rsvp, qos, quality of service, mpls, intserv, diffserv, multicast]